In [2]:
%run Paths_Notebook

StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 4, Finished, Available, Finished)

In [3]:
%run bronze_Notebook

StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b3ec307d-63cd-47c9-b9bb-250cce9013a7)

root
 |-- id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- birthdate: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- race: string (nullable = true)
 |-- department: string (nullable = true)
 |-- jobtitle: string (nullable = true)
 |-- location: string (nullable = true)
 |-- hire_date: string (nullable = true)
 |-- termdate: timestamp (nullable = true)
 |-- location_city: string (nullable = true)
 |-- location_state: string (nullable = true)

Number of rows  22227


SynapseWidget(Synapse.DataFrame, e909d10b-072d-440e-b047-5d993b0ec2c5)

SynapseWidget(Synapse.DataFrame, 0e1e792b-d26b-4f3c-b2a2-b6ea8a0f3650)

HR data successfully moved to the Bronze Layer: 'hr_data_bronze.parquet'


In [4]:
# Bronze Layer Data
bronze_df = spark.read.parquet(bronze_parquet_path)

# Show sample data
bronze_df.show(5)


StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 11, Finished, Available, Finished)

+----------+----------+---------+----------+------+--------------------+---------------+--------------------+------------+---------+-------------------+-------------+--------------+
|        id|first_name|last_name| birthdate|gender|                race|     department|            jobtitle|    location|hire_date|           termdate|location_city|location_state|
+----------+----------+---------+----------+------+--------------------+---------------+--------------------+------------+---------+-------------------+-------------+--------------+
|00-0381660|Wainwright| Corfield|12/13/1996|     M|               Asian|    Engineering|Business Systems ...|Headquarters|2/20/2001|2008-12-05 01:21:48|    Cleveland|          Ohio|
|00-1100714|     Regen|  Nafzger|  4/6/1990|     M|               White|Human Resources|    Senior Recruiter|Headquarters|7/23/2016|2022-07-12 02:26:01|    Cleveland|          Ohio|
|00-1268049|       Fay| Monnelly|  7/9/1966|     M|Native Hawaiian o...|    Engineering| S

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from delta.tables import DeltaTable

StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 12, Finished, Available, Finished)

In [6]:
# Step 1 Initialize Spark Session with Delta support
spark = SparkSession.builder \
    .appName("HR Silver Layer") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .enableHiveSupport() \
    .getOrCreate()

StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 13, Finished, Available, Finished)

In [7]:
# Step 2: Count Total Records and Null Values Before Processing
total_records_before = bronze_df
null_counts_before = {col_name: bronze_df.filter(col(col_name).isNull()).count() for col_name in bronze_df.columns}

StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 14, Finished, Available, Finished)

In [8]:
# Step 3: Remove Nulls (except termdate)
columns_to_clean = [col_name for col_name in bronze_df.columns if col_name != "termdate"]
df_cleaned = bronze_df.dropna(subset=columns_to_clean)
df_cleaned.show()


StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 15, Finished, Available, Finished)

+----------+----------+----------+----------+------+--------------------+--------------------+--------------------+------------+----------+-------------------+-------------+--------------+
|        id|first_name| last_name| birthdate|gender|                race|          department|            jobtitle|    location| hire_date|           termdate|location_city|location_state|
+----------+----------+----------+----------+------+--------------------+--------------------+--------------------+------------+----------+-------------------+-------------+--------------+
|00-0381660|Wainwright|  Corfield|12/13/1996|     M|               Asian|         Engineering|Business Systems ...|Headquarters| 2/20/2001|2008-12-05 01:21:48|    Cleveland|          Ohio|
|00-1100714|     Regen|   Nafzger|  4/6/1990|     M|               White|     Human Resources|    Senior Recruiter|Headquarters| 7/23/2016|2022-07-12 02:26:01|    Cleveland|          Ohio|
|00-1268049|       Fay|  Monnelly|  7/9/1966|     M|Nat

In [9]:
# Step 4: Remove Duplicates
df_silver = df_cleaned.dropDuplicates()
df_silver.show()

StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 16, Finished, Available, Finished)

+----------+----------+---------+----------+------+--------------------+--------------------+--------------------+------------+----------+-------------------+-------------+--------------+
|        id|first_name|last_name| birthdate|gender|                race|          department|            jobtitle|    location| hire_date|           termdate|location_city|location_state|
+----------+----------+---------+----------+------+--------------------+--------------------+--------------------+------------+----------+-------------------+-------------+--------------+
|10-8732936| Shurlocke| Merriott|12/15/1997|     M|               White|             Support|Help Desk Technician|Headquarters| 7/26/2018|2021-03-15 14:22:26|    Cleveland|          Ohio|
|14-4552526|    Emilio|   Mecchi| 9/20/1966|     M|Black or African ...|         Engineering|Systems Administr...|Headquarters| 10/1/2019|2039-05-07 18:36:59|    Cleveland|          Ohio|
|36-1975252|   Elonore|Mattinson| 7/30/1999|     M|         

In [10]:
# Step 5:  Null Value count
null_counts_before = {c: bronze_df.filter(col(c).isNull()).count() for c in bronze_df.columns}

StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 17, Finished, Available, Finished)

In [11]:
# Step 6: Count Total Records After Processing
total_records_before = bronze_df.count()

StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 18, Finished, Available, Finished)

In [12]:
# Step 7: Count after processing
total_records_after = df_silver.count()
nulls_removed = total_records_before - df_cleaned.count()  #  Get count before subtracting
duplicates_removed = df_cleaned.count() - total_records_after  #  Get count before subtracting

StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 19, Finished, Available, Finished)

In [13]:
# Step 8: Show summary log
print(f"""
Silver Layer Processing Summary:

Total records before processing: {total_records_before}
Null values removed (except termdate): {nulls_removed}
Duplicate records removed: {duplicates_removed}
Total records after cleaning: {total_records_after}
""")

StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 20, Finished, Available, Finished)


Silver Layer Processing Summary:

Total records before processing: 22227
Null values removed (except termdate): 0
Duplicate records removed: 13
Total records after cleaning: 22214



In [14]:
# Create a new column 'id_cleaned' by removing dashes from 'id'
from pyspark.sql.functions import *
cleaned_id = df_silver.withColumn("id", regexp_replace(col("id"), "-", ""))
silver_df = cleaned_id.withColumn("id", col("id").cast("int"))
# silver_df.select("id", "id_cleaned").show(10)
display(silver_df)

StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d172ad61-323a-4d75-a0a1-71e00eb1c843)

In [15]:
# Convert 'birthdate' and 'hire_date' into proper date format
silver_df_birth_date = silver_df.withColumn("birthdate", to_date(col("birthdate"), "M/d/yyyy"))
hr_data_silver = silver_df_birth_date.withColumn("hire_date",to_date(col("hire_date"),"M/d/yyyy"))
# display(silver_df_hire_date)
# display(silver_df_birth_date)
hr_data_silver.printSchema()

StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 22, Finished, Available, Finished)

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- birthdate: date (nullable = true)
 |-- gender: string (nullable = true)
 |-- race: string (nullable = true)
 |-- department: string (nullable = true)
 |-- jobtitle: string (nullable = true)
 |-- location: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- termdate: timestamp (nullable = true)
 |-- location_city: string (nullable = true)
 |-- location_state: string (nullable = true)



In [16]:
# Change id to employee_id

hr_data_silver = hr_data_silver.withColumnRenamed("id","employee_id") 
display(hr_data_silver)


StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e60000d7-f007-41c9-8b5d-cf81617fe814)

In [17]:
hr_data_silver.write.format("delta").mode("overwrite").save(silver_fpath)

StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 24, Finished, Available, Finished)

In [18]:
silver_path = r"abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/Silver_Lakehouse5.Lakehouse/Tables/hr_data_silver "
hr_data_silver.write.format("delta").mode("overwrite").save(silver_path)

test_df = spark.read.format("delta").load(silver_path)
display(test_df)

StatementMeta(, 7563a625-ee35-4f6b-8249-b5d1cdf2dddd, 25, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 08797414-a96e-4ff8-b480-58b52023e5fe)